In [1]:
from datasets import load_dataset
from transformers import ASTFeatureExtractor
import torch
from torch.utils.data import DataLoader
import torch.profiler
import numpy as np
from datasets import Audio


In [2]:
import platform
print(f"Python version: {platform.python_version()}")
print(f"System: {platform.system()} {platform.release()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA device count: {torch.cuda.device_count()}")

Python version: 3.12.7
System: Windows 11
PyTorch version: 2.7.1+cu118
CUDA available: True
CUDA version: 11.8
GPU: NVIDIA GeForce RTX 4050 Laptop GPU
CUDA device count: 1


In [3]:
dataset = load_dataset("audiofolder", data_dir="./dataset")

pretrained_model = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(pretrained_model, num_mel_bins=64, max_length=507)


model_input_name = feature_extractor.model_input_names[0]
SAMPLING_RATE = feature_extractor.sampling_rate
num_labels = len(np.unique(dataset["train"]["label"]))


Resolving data files:   0%|          | 0/253906 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31738 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31742 [00:00<?, ?it/s]

Computing checksums:  72%|#######1  | 182069/253906 [00:05<00:01, 36412.44it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
print('dataset:', dataset)
print('dataset train features:', dataset['train'].features)
print('dataset test features:', dataset['test'].features)
print('dataset validation features:', dataset['validation'].features)
print('model_input_name:', model_input_name)
print('SAMPLING_RATE:', SAMPLING_RATE)
print('dataset["train"][0]:', dataset['train'][0])
print('num_labels:', num_labels)
print('dataset columns:', dataset['train'].column_names)


dataset: DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 253906
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 31738
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 31742
    })
})
dataset train features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
dataset test features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
dataset validation features: {'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'label': ClassLabel(names=['fake', 'real'], id=None)}
model_input_name: input_values
SAMPLING_RATE: 16000
dataset["train"][0]: {'audio': {'path': 'D:\\Documents\\Github\\thesis-testing\\dataset\\train\\fake\\0.wav', 'array': array([ 8.55924794e-04,  5.84704467e-05,  7.75483320e-04, ...,
       -1.0269136

In [6]:
# calculate values for normalization
feature_extractor.do_normalize = False

# Initialize running statistics
n = 0
mean = 0.0
M2 = 0.0  # For running variance calculation

def preprocess_audio(batch):
    wavs = [audio["array"] for audio in batch["input_values"]]
    inputs = feature_extractor(wavs, sampling_rate=SAMPLING_RATE, return_tensors="pt", return_attention_mask=True, max_length=507)
    return {
        model_input_name: inputs.get(model_input_name),
        "labels": torch.tensor(batch["label"])
    }

dataset = dataset.rename_column("audio", "input_values")
dataset["train"].set_transform(preprocess_audio, output_all_columns=False)

# Process in batches to save memory
for batch in dataset["train"]:

    audio_input = batch[model_input_name]
    batch_size = audio_input.shape[0]

    # Calculate batch statistics
    batch_mean = torch.mean(audio_input)
    batch_variance = torch.var(audio_input, unbiased=False)  # Use N instead of N-1 for population variance

    # Update running statistics using Welford's online algorithm
    delta = batch_mean - mean
    mean += delta * batch_size / (n + batch_size)
    M2 += batch_variance * batch_size + delta ** 2 * n * batch_size / (n + batch_size)
    n += batch_size

# Calculate final statistics
feature_extractor.mean = mean.item()
feature_extractor.std = torch.sqrt(M2 / n).item()  # Population standard deviation
feature_extractor.do_normalize = True

print('mean: ', feature_extractor.mean)
print('std: ', feature_extractor.std)

mean:  -7.9208784103393555
std:  5.350068092346191


In [7]:
from audiomentations import Compose, AddGaussianSNR, GainTransition, Gain, ClippingDistortion, TimeStretch, PitchShift

audio_augmentations = Compose([
    AddGaussianSNR(min_snr_db=10, max_snr_db=20),
    Gain(min_gain_db=-6, max_gain_db=6),
    GainTransition(min_gain_db=-6, max_gain_db=6, min_duration=0.01, max_duration=0.3, duration_unit="fraction"),
    ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=30, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2),
    PitchShift(min_semitones=-4, max_semitones=4),
], p=0.8, shuffle=True)

In [8]:
def preprocess_audio_with_transforms(batch):
    # we apply augmentations on each waveform
    wavs = [audio_augmentations(audio["array"], sample_rate=SAMPLING_RATE) for audio in batch["input_values"]]
    inputs = feature_extractor(wavs, sampling_rate=SAMPLING_RATE, return_tensors="pt", return_attention_mask=True, max_length=507)
    return {
        model_input_name: inputs.get(model_input_name),
        "labels": torch.tensor(batch["label"])
    }

print(dataset)
# Cast the audio column to the appropriate feature type and rename it
dataset = dataset.cast_column("input_values", Audio(sampling_rate=feature_extractor.sampling_rate))



DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 253906
    })
    validation: Dataset({
        features: ['input_values', 'label'],
        num_rows: 31738
    })
    test: Dataset({
        features: ['input_values', 'label'],
        num_rows: 31742
    })
})


In [9]:
# with augmentations on the training set
dataset["train"].set_transform(preprocess_audio_with_transforms, output_all_columns=False)
# w/o augmentations on the test set
dataset["test"].set_transform(preprocess_audio, output_all_columns=False)
dataset["validation"].set_transform(preprocess_audio, output_all_columns=False)


In [10]:
temp_ds = dataset['train'].with_format(None)  # This removes any applied transforms
temp_ds_test = dataset['test'].with_format(None)  # This removes any applied transforms

print(temp_ds[0])
unique_labels = sorted(set(temp_ds["label"] + temp_ds_test["label"]))
print(unique_labels)

{'input_values': {'path': 'D:\\Documents\\Github\\thesis-testing\\dataset\\train\\fake\\0.wav', 'array': array([ 8.55924794e-04,  5.84704467e-05,  7.75483320e-04, ...,
       -1.02691360e-04, -2.82649242e-04,  0.00000000e+00]), 'sampling_rate': 16000}, 'label': 0}
[0, 1]


In [11]:
from transformers import ASTConfig, ASTForAudioClassification

# Load configuration from the pretrained model
config = ASTConfig.from_pretrained(pretrained_model)

# Update configuration with the number of labels in our dataset
config.num_mel_bins = 64  # Make sure this matches your feature extractor
config.max_length = 507   # Or whatever your sequence length is
config.num_labels = num_labels
config.label2id = {"fake": 0, "real": 1}
config.id2label = {0: "fake", 1: "real"}

# Initialize the model with the updated configuration
model = ASTForAudioClassification.from_pretrained(
    pretrained_model,
    config=config,
    ignore_mismatched_sizes=True
)
model.init_weights()

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- audio_spectrogram_transformer.embeddings.position_embeddings: found shape torch.Size([1, 1214, 768]) in the checkpoint and torch.Size([1, 252, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

# Configure training run with TrainingArguments class
training_args = TrainingArguments(
    output_dir="./runs/ast_classifier",
    logging_dir="./logs/ast_classifier",
    report_to="tensorboard",
    learning_rate=5e-5,
    num_train_epochs=10,
    per_device_train_batch_size=8,  # Start with 8, can increase if memory allows
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    dataloader_num_workers=0,  # Changed from 4 to 0 for Windows
    dataloader_pin_memory=False,  # Disable pin_memory on Windows
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=20,
    remove_unused_columns=False,
    fp16=True,
    fp16_full_eval=True,
    no_cuda=not torch.cuda.is_available(),
)

In [13]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

AVERAGE = "macro" if config.num_labels > 2 else "binary"

def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    predictions = np.argmax(logits, axis=1)
    metrics = accuracy.compute(predictions=predictions, references=eval_pred.label_ids)
    metrics.update(precision.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    metrics.update(recall.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    metrics.update(f1.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    return metrics

In [14]:
from transformers import Trainer, DataCollatorWithPadding
from tqdm.auto import tqdm

# Initialize the data collator
data_collator = DataCollatorWithPadding(
    tokenizer=feature_extractor,  # Your feature extractor acts as the tokenizer
    padding=True,
    max_length=config.max_length,
    return_tensors="pt"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
    data_collator=data_collator,
)

C:\Users\kace\AppData\Local\Temp\ipykernel_14296\3091343712.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# Add this before training to verify shapes
sample = next(iter(dataset["train"]))
print(f"Sample input shape: {sample['input_values'].shape}")

# Forward pass to check for errors
with torch.no_grad():
    output = model(sample["input_values"].unsqueeze(0))
print("Forward pass successful!")

d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Sample input shape: torch.Size([507, 64])
Forward pass successful!


In [16]:
print(f"Model config: {model.config}")
print(f"Sample input shape: {sample['input_values'].shape}")
print(f"Model's expected input shape: {model.config.max_length}")
print(f"Feature extractor config: {feature_extractor}")

Model config: ASTConfig {
  "architectures": [
    "ASTForAudioClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "frequency_stride": 10,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "fake",
    "1": "real"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "fake": 0,
    "real": 1
  },
  "layer_norm_eps": 1e-12,
  "max_length": 507,
  "model_type": "audio-spectrogram-transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_mel_bins": 64,
  "patch_size": 16,
  "qkv_bias": true,
  "time_stride": 10,
  "torch_dtype": "float32",
  "transformers_version": "4.53.1"
}

Sample input shape: torch.Size([507, 64])
Model's expected input shape: 507
Feature extractor config: ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 507,
  "mean": -7.9208784103393555,
  "num_mel_bins": 64,
  "padding_side":

In [17]:
# Check GPU availability
print(f"Using device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
     print(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
     print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
     print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0)/1e9:.2f} GB")


Using device: NVIDIA GeForce RTX 4050 Laptop GPU
GPU Memory Total: 6.44 GB
GPU Memory Allocated: 0.00 GB
GPU Memory Cached: 0.00 GB


In [18]:
trainer.train(resume_from_checkpoint=False) #Set to False when no checkpoint is available.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.070400,0.017584,0.994739,0.990321,0.999244,0.994762
2,0.041000,0.004508,0.998834,0.998740,0.998929,0.998834
3,0.010400,0.007480,0.998110,0.996795,0.999433,0.998112
4,0.011200,0.003295,0.999181,0.998615,0.999748,0.999181
5,0.018700,0.003409,0.999401,0.999559,0.999244,0.999401
6,0.013800,0.002013,0.999401,0.999559,0.999244,0.999401
7,0.006000,0.001036,0.999779,0.999748,0.999811,0.999779
8,0.006700,0.002227,0.999370,0.999622,0.999118,0.999370
9,0.000600,0.001330,0.999779,0.999748,0.999811,0.999779
10,0.000100,0.001003,0.999874,0.999811,0.999937,0.999874


NaN or Inf found in input tensor.
d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
NaN or Inf found in input tensor.
d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
d:\Documents\Github\thesis-testing\venv\Lib\site-packages\audiomentations\core\transforms_interface.py:1

TrainOutput(global_step=158700, training_loss=0.02415744743147359, metrics={'train_runtime': 107030.1665, 'train_samples_per_second': 23.723, 'train_steps_per_second': 1.483, 'total_flos': 4.2240782021685215e+19, 'train_loss': 0.02415744743147359, 'epoch': 10.0})

In [19]:
if torch.cuda.is_available():
     print(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
     print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
     print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0)/1e9:.2f} GB")
else:
    print("No GPU available")

GPU Memory Total: 6.44 GB
GPU Memory Allocated: 1.06 GB
GPU Memory Cached: 2.63 GB


In [55]:
##### MODEL EVALUATION AND PREDICTION

import os
import json

# Define the checkpoint path
checkpoint_path = "runs/ast_classifier/checkpoint-158700"

# Load the model and feature extractor
model = ASTForAudioClassification.from_pretrained(checkpoint_path)
feature_extractor = ASTFeatureExtractor.from_pretrained(checkpoint_path)

# Look for training history
trainer_state_path = os.path.join(checkpoint_path, "trainer_state.json")
if os.path.exists(trainer_state_path):
    with open(trainer_state_path, "r") as f:
        trainer_state = json.load(f)
    print("\nTraining metrics from trainer_state.json:")
    print(json.dumps(trainer_state, indent=2))

# Set model to evaluation mode
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model loaded on {device}")

import librosa
def predict_audio(file_path, model, feature_extractor, device="cuda"):
    # Load audio file
    audio, sr = librosa.load(file_path, sr=feature_extractor.sampling_rate)

    # Preprocess the audio
    inputs = feature_extractor(
        audio,
        sampling_rate=feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)

    print(f"Raw logits: {logits}")
    print(f"Raw probabilities: {probabilities}")

    # Get predicted class (0 for fake, 1 for real)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][predicted_class].item()

    print(f"P(fake): {probabilities[0][0].item():.4f}")
    print(f"P(real): {probabilities[0][1].item():.4f}")

    # Map class index to label
    label = "fake" if predicted_class == 0 else "real"

    return {
        "label": label,
        "confidence": confidence,
        "probabilities": {
            "fake": probabilities[0][0].item(),
            "real": probabilities[0][1].item()
        }
    }

audio_file_path = "./cut/HitlerAI(10).wav" # Adjust the path as needed
result = predict_audio(audio_file_path, model, feature_extractor, device)

print(f"Prediction: {result['label']}")
print(f"Confidence: {result['confidence']:.4f}")
print(f"Probabilities - Fake: {result['probabilities']['fake']:.4f}, Real: {result['probabilities']['real']:.4f}")


Training metrics from trainer_state.json:
{
  "best_global_step": 158700,
  "best_metric": 0.9998739839959675,
  "best_model_checkpoint": "./runs/ast_classifier\\checkpoint-158700",
  "epoch": 10.0,
  "eval_steps": 500,
  "global_step": 158700,
  "is_hyper_param_search": false,
  "is_local_process_zero": true,
  "is_world_process_zero": true,
  "log_history": [
    {
      "epoch": 0.0012602791518321309,
      "grad_norm": 6.989597320556641,
      "learning_rate": 4.9994328922495275e-05,
      "loss": 0.876,
      "step": 20
    },
    {
      "epoch": 0.0025205583036642617,
      "grad_norm": 8.377490043640137,
      "learning_rate": 4.9988027725267804e-05,
      "loss": 0.3318,
      "step": 40
    },
    {
      "epoch": 0.0037808374554963923,
      "grad_norm": 5.075488090515137,
      "learning_rate": 4.9981726528040326e-05,
      "loss": 0.2956,
      "step": 60
    },
    {
      "epoch": 0.005041116607328523,
      "grad_norm": 5.285743236541748,
      "learning_rate": 4.99754

In [54]:
#### AUDIO CUTTING SCRIPT

import os
import sys
import librosa
import soundfile as sf

def cut_audio(input_path, output_dir="./cut", clip_duration=5):
     # Ensure output directory exists
     os.makedirs(output_dir, exist_ok=True)

     # Get file name and extension
     base_name = os.path.basename(input_path)
     name, ext = os.path.splitext(base_name)

     # Load audio
     audio, sr = librosa.load(input_path, sr=None)
     total_duration = librosa.get_duration(y=audio, sr=sr)
     clip_samples = int(clip_duration * sr)
     num_clips = int(total_duration // clip_duration) + (1 if total_duration % clip_duration > 0 else 0)

     for i in range(num_clips):
          start_sample = i * clip_samples
          end_sample = min((i + 1) * clip_samples, len(audio))
          clip_audio = audio[start_sample:end_sample]
          out_path = os.path.join(output_dir, f"{name}({i+1}){ext}")
          sf.write(out_path, clip_audio, sr)
          print(f"Saved: {out_path}")

if __name__ == "__main__":
     if len(sys.argv) < 2:
          print("Usage: python cut_audio.py <path_to_audio_file>")
          sys.exit(1)
     audio_path = "./dataset/HitlerAI.wav"  # Adjust the path as needed
     if not os.path.exists(audio_path):
          print(f"File not found: {audio_path}")
          sys.exit(1)
     cut_audio(audio_path)

Saved: ./cut\HitlerAI(1).wav
Saved: ./cut\HitlerAI(2).wav
Saved: ./cut\HitlerAI(3).wav
Saved: ./cut\HitlerAI(4).wav
Saved: ./cut\HitlerAI(5).wav
Saved: ./cut\HitlerAI(6).wav
Saved: ./cut\HitlerAI(7).wav
Saved: ./cut\HitlerAI(8).wav
Saved: ./cut\HitlerAI(9).wav
Saved: ./cut\HitlerAI(10).wav
Saved: ./cut\HitlerAI(11).wav
Saved: ./cut\HitlerAI(12).wav
Saved: ./cut\HitlerAI(13).wav
Saved: ./cut\HitlerAI(14).wav
Saved: ./cut\HitlerAI(15).wav
Saved: ./cut\HitlerAI(16).wav
Saved: ./cut\HitlerAI(17).wav
Saved: ./cut\HitlerAI(18).wav
Saved: ./cut\HitlerAI(19).wav
Saved: ./cut\HitlerAI(20).wav
Saved: ./cut\HitlerAI(21).wav
Saved: ./cut\HitlerAI(22).wav
Saved: ./cut\HitlerAI(23).wav
Saved: ./cut\HitlerAI(24).wav
Saved: ./cut\HitlerAI(25).wav
Saved: ./cut\HitlerAI(26).wav
